In [1]:
import re
import torch.nn as nn


In [87]:
import torch
import torch.nn as nn
import timm


c:\Users\preet\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:

class Block(nn.Module):

    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, 1, 1, 0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride, 1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, 1,1, 0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        # self.maxpool = nn.MaxPool2d(3, 2, 1)
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        # out = self.maxpool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        
        out += identity
        out = self.relu(out)
        return out
    

class Resnet(nn.Module):
    def __init__(self, Block, layers, image_channels, num_classes):
        super(Resnet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, self.in_channels, 7, 2, 3)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(Block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(Block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(Block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(Block, layers[3], out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*4, num_classes)

        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)

        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)

        return out

    def _make_layer(self,Block,num_res_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != out_channels*4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels*4))
        
        layers.append(Block(self.in_channels, out_channels, identity_downsample=identity_downsample, stride=stride))
        self.in_channels = out_channels*4

        for i in range(num_res_blocks-1):
            layers.append(Block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

In [72]:
def ResNet50(img_channels=3, num_classes=1000):
    return Resnet(Block, [3, 4, 6, 3], img_channels, num_classes)

net = ResNet50()

In [63]:

x = torch.randn(5, 3, 224, 224)
identity_downsample = nn.Sequential(nn.Conv2d(3, 64, kernel_size=1, stride=1), nn.BatchNorm2d(64))

identity_downsample(x).shape

torch.Size([5, 64, 224, 224])

In [34]:
b = Block(3, 16, stride=1, identity_downsample=identity_downsample)
b(x).shape

torch.Size([5, 64, 224, 224])

In [74]:
net(x).shape

torch.Size([5, 1000])

In [83]:
sum([param.nelement() for param in net.parameters()])

25583592

In [86]:
list(net.children())[-1]

Linear(in_features=2048, out_features=1000, bias=True)

In [73]:
t1 = torch.randn(256, 56, 56)
t2 = torch.randn(256, 28, 28)
maxpool = nn.MaxPool2d(3, 2, 1)
t1 = maxpool(t1)
t1.shape, t2.shape

(torch.Size([256, 28, 28]), torch.Size([256, 28, 28]))

In [1]:

print(2.23123//1)

2.0
